In [ ]:
import os, glob, time, h5py, warnings, sys
import os.path
import torch
import multiprocessing as mp
import matplotlib.pyplot as plt   # plots
import numpy as np
import pandas as pd
import scipy.sparse as sp
from scipy.optimize import curve_fit

import random
#from scipy.stats import gaussian_kde
#from sklearn.neighbors import NearestNeighbors
#from scipy.special import digamma
from math import log, pi, gamma

from liblibra_core import *
import util.libutil as comn

from commons import kozachenko_leonenko_entropy
from commons import show_hist

import libra_py
from libra_py import units
import libra_py.data_visualize
warnings.filterwarnings('ignore')

plt.rc('axes', titlesize=40)      # fontsize of the axes title
plt.rc('axes', labelsize=40)      # fontsize of the x and y labels
plt.rc('legend', fontsize=40)     # legend fontsize
plt.rc('xtick', labelsize=40)     # fontsize of the tick labels
plt.rc('ytick', labelsize=40)     # fontsize of the tick labels

plt.rc('figure.subplot', left=0.2)
plt.rc('figure.subplot', right=0.95)
plt.rc('figure.subplot', bottom=0.13)
plt.rc('figure.subplot', top=0.88)

font = {'family': 'serif',
        'color':  'blue',
        'weight': 'bold',
        'size': 36,
        }

# 1. The Complexity Metrics for the "Test-Cases"

We consider 6 choices of initial conditions (below) and 2 model Hamiltonian (Tully 1 and Tully 2)

For each: 

- we conduct quantum dynamics calculations and plot populations and coherences;
- compute the complexity metrics via Shannon and Kozachenko-Leonenko entropies 

In [ ]:
#======== Initial conditions and models ==============
iconds1 = []
#                 q0    p0   istate
iconds1.append( ["-6.0",  "20.0",   0] )
iconds1.append( ["-6.0",  "20.0",   1] )
iconds1.append( ["-10.0", "20.0",   0] )
iconds1.append( ["-10.0", "20.0",   1] )
iconds1.append( ["-6.0",  "10.0",   0] )
iconds1.append( ["-6.0",  "10.0",   1] )

## 1.1. Plot quantum populations and coherences
Function to plot populations and coherence of the given exact simulations:

In [ ]:
def plot_pop_coherences(model, iconds):
    
    for c, icond in enumerate(iconds):    
        q0, p0, istate = icond[0], icond[1], icond[2]
    
        #================= Adiabatic SE populations ================
        fig, ax = plt.subplots(2, 1, figsize=(16, 18), sharex="col")
    
        prf = F"EXACT-model_{model}-istate_{istate}-q0_{q0}-p0_{p0}"   
    
        f = torch.load(F"{prf}/data.pt")
        t = torch.tensor(f["time"]) * units.au2fs  # to convert to fs
        rho_dia = torch.tensor(f["rho_dia_all"])
        rho_adi = torch.tensor(f["rho_adi_all"])

    
        ax[0].plot(t[:], rho_adi[:, 0, 0], label=r"$\rho^{adi}_{00}$", lw=8, color = "black")
        ax[0].plot(t[:], rho_adi[:, 1, 1], label=r"$\rho^{adi}_{11}$", lw=8, color = "blue")
        ax[0].plot(t[:], rho_adi[:, 0, 1].real, label=r"$Re[\rho^{adi}_{01}]$", lw=8, color = "green")
        ax[0].plot(t[:], rho_adi[:, 0, 1].imag, label=r"$Im[\rho^{adi}_{01}]$", lw=8, color = "red")
        ax[0].plot(t[:], rho_adi[:, 1, 0].real, label=r"$Re[\rho^{adi}_{10}]$", lw=8, color = "green", ls="--")
        ax[0].plot(t[:], rho_adi[:, 1, 0].imag, label=r"$Im[\rho^{adi}_{10}]$", lw=8, color = "red", ls="--")
        ax[0].legend()

        ax[1].plot(t[:], rho_dia[:, 0, 0], label=r"$\rho^{dia}_{00}$", lw=8, color = "black")
        ax[1].plot(t[:], rho_dia[:, 1, 1], label=r"$\rho^{dia}_{11}$", lw=8, color = "blue")
        ax[1].plot(t[:], rho_dia[:, 0, 1].real, label=r"$Re[\rho^{dia}_{01}]$", lw=8, color = "green")
        ax[1].plot(t[:], rho_dia[:, 0, 1].imag, label=r"$Im[\rho^{dia}_{01}]$", lw=8, color = "red")
        ax[1].plot(t[:], rho_dia[:, 1, 0].real, label=r"$Re[\rho^{dia}_{10}]$", lw=8, color = "green", ls="--")
        ax[1].plot(t[:], rho_dia[:, 1, 0].imag, label=r"$Im[\rho^{dia}_{10}]$", lw=8, color = "red", ls="--")
        ax[1].legend()

        ax[0].set_ylabel("Adiabatic")
        ax[1].set_ylabel("Diabatic")
        
        fig.suptitle(F'MDL = {model} istate = {istate} $q_0$ = {q0} $p_0$ ={p0}', size=38) 
        fig.supxlabel("Time, fs", size=40)    
        plt.legend()
        plt.tight_layout()    
        plt.savefig(F'pop-coherence-{prf}.png')

### Tully 1

In [ ]:
plot_pop_coherences("Tully1", iconds1)

### Tully 2

In [ ]:
plot_pop_coherences("Tully2", iconds1)

## 1.2. Entropy - the measure of the complexity of quantum dynamics

Characterize the complexity of quantum dynamics - the measure of the model's complexity.

Here, we compute information (entropy) via several methods. We plot the corresponding distributions too.

In [ ]:
def plot_information(model, iconds, _bins=200, _sigma=0.0075):

    all_df = None
    for c, icond in enumerate(iconds):    
        q0, p0, istate = icond[0], icond[1], icond[2]
    
        #================= Adiabatic SE populations ================
        fig, ax = plt.subplots(2, 1, figsize=(18, 18), sharex="col")
    
        prf = F"EXACT-model_{model}-istate_{istate}-q0_{q0}-p0_{p0}"   
    
        f = torch.load(F"{prf}/data.pt")
        t = torch.tensor(f["time"]) * units.au2fs  # to convert to fs
        rho_dia = torch.tensor(f["rho_dia_all"])
        rho_adi = torch.tensor(f["rho_adi_all"])
    
        print(prf)
        print("Adiabatic dynamics")
        z = []
        z = z + list(np.array(rho_adi[:,0,0].real))
        z = z + list(np.array(rho_adi[:,1,1].real))
        z = z + list(np.array(rho_adi[:,0,1].real))
        z = z + list(np.array(rho_adi[:,0,1].imag))
        z = z + list(np.array(rho_adi[:,1,0].real))
        z = z + list(np.array(rho_adi[:,1,0].imag))
        v01, v02, v03 = show_hist(ax[0], z, _bins, _sigma)

        print("Diabatic dynamics")
        z = []
        z = z + list(np.array(rho_dia[:,0,0].real))
        z = z + list(np.array(rho_dia[:,1,1].real))
        z = z + list(np.array(rho_dia[:,0,1].real))
        z = z + list(np.array(rho_dia[:,0,1].imag))
        z = z + list(np.array(rho_dia[:,1,0].real))
        z = z + list(np.array(rho_dia[:,1,0].imag))
        v11, v12, v13 = show_hist(ax[1], z, _bins, _sigma)
        
        df = pd.DataFrame({ "p0": [p0],
                            "q0": [q0],
                            "istate":[istate],
                            "model":[model],
                            "adi, hist inf": [v01],
                            "adi, KDE inf": [v02],
                            "adi, KL inf": [v03],
                            "dia, hist inf": [v11],
                            "dia, KDE inf": [v12],
                            "dia, KL inf": [v13]
        })
        if all_df is None:
            all_df = df
        else:
            all_df = pd.concat([all_df, df], ignore_index=True)
                
    
        fig.suptitle(F'MDL = {model} istate = {istate} $q_0$ = {q0} $p_0$ ={p0}', size=38) 
        fig.supxlabel("Population", size=38)    
        plt.legend()
        plt.tight_layout()    
        plt.savefig(F'entropy-{prf}.png')
        
    return all_df

### Tully 1

In [ ]:
Tully1 = plot_information("Tully1", iconds1)

### Tully 2

In [ ]:
Tully2 = plot_information("Tully2", iconds1)

Combine the metrics and display the tables

In [ ]:
data_ex = pd.concat([Tully1, Tully2], ignore_index=True)

display(data_ex)

In [ ]:
data_ex.to_csv("metrics_Tully_1_2_exact.csv", index=False)

# 2. Compute Different Error Metrics for Methods

We consider several NA-MD methods - FSSH with different types of velocity rescaling and reversal-on-frustrated-hop options, and one option for decoherence-corrected IDA method. 

In [ ]:
methods = []
methods.append( "FSSH_g_minus" )
methods.append( "FSSH_g_plus" )
methods.append( "FSSH_g_jt" )
methods.append( "IDA_FSSH_g_minus" )
methods.append( "FSSH_h_minus" )
methods.append( "FSSH_v_minus" )

## 2.1. Computing and Plotting Error Measures of Different Methods

Just plotting the deviations of the approximate populations from the reference ones

- `plot_errors` - plot just populations
- `plot_errors2` - also coherences

In [ ]:
def plot_errors(model, iconds, methods):
    for c, icond in enumerate(iconds):    
        q0, p0, istate = icond[0], icond[1], icond[2]
    
        #================= Exact reference ================
        fig, ax = plt.subplots(3, 1, figsize=(16, 18), sharex="col")
    
        prf = F"EXACT-model_{model}-istate_{istate}-q0_{q0}-p0_{p0}"   
    
        f = torch.load(F"{prf}/data.pt")
        t = torch.tensor(f["time"]) * units.au2fs  # to convert to fs
        rho_dia = np.array(torch.tensor(f["rho_dia_all"]))
        rho_adi = np.array(torch.tensor(f["rho_adi_all"]))
    
        ax[0].set_ylabel("SE, adi")
        ax[1].set_ylabel("SH, adi")
        ax[2].set_ylabel("SE, dia")        
        
        for c2, method in enumerate(methods):
            prf = F"NAMD-model_{model}-istate_{istate}-q0_{q0}-p0_{p0}-method_{method}"
    
            with h5py.File(F"{prf}/mem_data.hdf", 'r') as F:
                pop0_se_adi = abs(np.array(F['se_pop_adi/data'][:,0])) - abs(rho_adi[:,0,0])
                pop0_sh_adi = abs(np.array(F['sh_pop_adi/data'][:,0])) - abs(rho_adi[:,0,0])
                pop0_se_dia = abs(np.array(F['se_pop_dia/data'][:,0])) - abs(rho_dia[:,0,0])
                
                pop1_se_adi = abs(np.array(F['se_pop_adi/data'][:,1])) - abs(rho_adi[:,1,1])
                pop1_sh_adi = abs(np.array(F['sh_pop_adi/data'][:,1])) - abs(rho_adi[:,1,1])
                pop1_se_dia = abs(np.array(F['se_pop_dia/data'][:,1])) - abs(rho_dia[:,1,1])

                
                coh_adi = np.array(F['coherence_adi/data'][:,0, 0]) - abs(rho_adi[:,0,0])**2
                coh_dia = np.array(F['coherence_dia/data'][:,0, 0]) - abs(rho_dia[:,0,0])**2

                t = np.array(F['time/data'][:]) * units.au2fs
            
                clrs_index = libra_py.data_visualize.clrs_index[ c2%10 ]
                ax[0].plot(t, pop0_se_adi, color=libra_py.data_visualize.colors[ clrs_index ], label=F"{method}", lw=5)
                ax[1].plot(t, pop0_sh_adi, color=libra_py.data_visualize.colors[ clrs_index ], label=F"{method}", lw=5)
                ax[2].plot(t, pop0_se_dia, color=libra_py.data_visualize.colors[ clrs_index ], label=F"{method}", lw=5)

                ax[0].plot(t, pop1_se_adi, color=libra_py.data_visualize.colors[ clrs_index ], label=F"{method}", lw=5, ls="--")
                ax[1].plot(t, pop1_sh_adi, color=libra_py.data_visualize.colors[ clrs_index ], label=F"{method}", lw=5, ls="--")
                ax[2].plot(t, pop1_se_dia, color=libra_py.data_visualize.colors[ clrs_index ], label=F"{method}", lw=5, ls="--")

        fig.suptitle(F'MDL = {model} istate = {istate} $q_0$ = {q0} $p_0$ ={p0}', size=38) 
        fig.supxlabel("Time, fs", size=40)    
        plt.legend()
        plt.tight_layout()    
        plt.savefig(F'error-Populations_model_{model}_method_{method}_istate{istate}_q0{q0}_p0{p0}.png')

In [ ]:
def plot_errors2(model, iconds, methods):
    """
    Also add plotting of errors in coherences
    """
    for c, icond in enumerate(iconds):    
        q0, p0, istate = icond[0], icond[1], icond[2]
    
        #================= Exact reference ================
        fig, ax = plt.subplots(3, 1, figsize=(16, 18), sharex="col")
    
        prf = F"EXACT-model_{model}-istate_{istate}-q0_{q0}-p0_{p0}"   
    
        f = torch.load(F"{prf}/data.pt")
        t = torch.tensor(f["time"]) * units.au2fs  # to convert to fs
        rho_dia = np.array(torch.tensor(f["rho_dia_all"]))
        rho_adi = np.array(torch.tensor(f["rho_adi_all"]))
    
        ax[0].set_ylabel("SE, adi")
        ax[1].set_ylabel("SH, adi")
        ax[2].set_ylabel("SE, dia")        
        
        for c2, method in enumerate(methods):
            prf = F"NAMD-model_{model}-istate_{istate}-q0_{q0}-p0_{p0}-method_{method}"
    
            with h5py.File(F"{prf}/mem_data.hdf", 'r') as F:
                pop0_se_adi = abs(np.array(F['se_pop_adi/data'][:,0])) - abs(rho_adi[:,0,0])
                pop0_sh_adi = abs(np.array(F['sh_pop_adi/data'][:,0])) - abs(rho_adi[:,0,0])
                pop0_se_dia = abs(np.array(F['se_pop_dia/data'][:,0])) - abs(rho_dia[:,0,0])
                
                pop1_se_adi = abs(np.array(F['se_pop_adi/data'][:,1])) - abs(rho_adi[:,1,1])
                pop1_sh_adi = abs(np.array(F['sh_pop_adi/data'][:,1])) - abs(rho_adi[:,1,1])
                pop1_se_dia = abs(np.array(F['se_pop_dia/data'][:,1])) - abs(rho_dia[:,1,1])

                
                coh_adi01 = np.array(F['coherence_adi/data'][:,0,1]) - abs(rho_adi[:,0,1])**2
                coh_adi10 = np.array(F['coherence_adi/data'][:,1,0]) - abs(rho_adi[:,1,0])**2
                coh_dia01 = np.array(F['coherence_dia/data'][:,0,1]) - abs(rho_dia[:,0,1])**2
                coh_dia10 = np.array(F['coherence_dia/data'][:,1,0]) - abs(rho_dia[:,1,0])**2
                
                t = np.array(F['time/data'][:]) * units.au2fs
            
                clrs_index = libra_py.data_visualize.clrs_index[ c2%10 ]
                clr = libra_py.data_visualize.colors[ clrs_index ]
                
                # SE, adi                
                ax[0].plot(t, pop0_se_adi, color=clr, label=F"{method}", lw=5)
                ax[0].plot(t, pop1_se_adi, color=clr, label="", lw=5, ls="--")
                ax[0].plot(t, coh_adi01, color=clr, label="", lw=5, ls='', marker='o')
                ax[0].plot(t, coh_adi10, color=clr, label="", lw=5, ls='', marker='*')

                # SH, adi                
                ax[1].plot(t, pop0_sh_adi, color=clr, label=F"{method}", lw=5)
                ax[1].plot(t, pop1_sh_adi, color=clr, label="", lw=5, ls="--")
                ax[1].plot(t, coh_adi01, color=clr, label="", lw=5, ls='', marker='o')
                ax[1].plot(t, coh_adi10, color=clr, label="", lw=5, ls='', marker='*')
                
                # SE, dia                
                ax[2].plot(t, pop0_se_dia, color=clr, label=F"{method}", lw=5)
                ax[2].plot(t, pop1_se_dia, color=clr, label="", lw=5, ls="--")
                ax[2].plot(t, coh_dia01, color=clr, label="", lw=5, ls='', marker='o')
                ax[2].plot(t, coh_dia10, color=clr, label="", lw=5, ls='', marker='*')

        fig.suptitle(F'MDL = {model} istate = {istate} $q_0$ = {q0} $p_0$ ={p0}', size=38) 
        fig.supxlabel("Time, fs", size=40)    
        plt.legend()
        plt.tight_layout()    
        plt.savefig(F'error-Populations_model_{model}_method_{method}_istate{istate}_q0{q0}_p0{p0}.png')

In [ ]:
def plot_errors3(model, iconds, methods):
    """
    Also add plotting of errors in coherences
    """
    for c, icond in enumerate(iconds):    
        q0, p0, istate = icond[0], icond[1], icond[2]
    
        #================= Exact reference ================
        fig, ax = plt.subplots(3, 1, figsize=(16, 18), sharex="col")
    
        prf = F"EXACT-model_{model}-istate_{istate}-q0_{q0}-p0_{p0}"   
    
        f = torch.load(F"{prf}/data.pt")
        t = torch.tensor(f["time"]) * units.au2fs  # to convert to fs
        rho_dia = np.array(torch.tensor(f["rho_dia_all"]))
        rho_adi = np.array(torch.tensor(f["rho_adi_all"]))
    
        ax[0].set_ylabel("SE, adi")
        ax[1].set_ylabel("SH, adi")
        ax[2].set_ylabel("SE, dia")        
        
        for c2, method in enumerate(methods):
            prf = F"NAMD-model_{model}-istate_{istate}-q0_{q0}-p0_{p0}-method_{method}"
    
            with h5py.File(F"{prf}/mem_data.hdf", 'r') as F:
                pop0_se_adi = abs(np.array(F['se_pop_adi/data'][:,0]))**2 - abs(rho_adi[:,0,0])**2
                pop0_sh_adi = abs(np.array(F['sh_pop_adi/data'][:,0]))**2 - abs(rho_adi[:,0,0])**2
                pop0_se_dia = abs(np.array(F['se_pop_dia/data'][:,0]))**2 - abs(rho_dia[:,0,0])**2
                
                pop1_se_adi = abs(np.array(F['se_pop_adi/data'][:,1]))**2 - abs(rho_adi[:,1,1])**2
                pop1_sh_adi = abs(np.array(F['sh_pop_adi/data'][:,1]))**2 - abs(rho_adi[:,1,1])**2
                pop1_se_dia = abs(np.array(F['se_pop_dia/data'][:,1]))**2 - abs(rho_dia[:,1,1])**2

                
                coh_adi01 = np.array(F['coherence_adi/data'][:,0,1]) - abs(rho_adi[:,0,1])**2
                coh_adi10 = np.array(F['coherence_adi/data'][:,1,0]) - abs(rho_adi[:,1,0])**2
                coh_dia01 = np.array(F['coherence_dia/data'][:,0,1]) - abs(rho_dia[:,0,1])**2
                coh_dia10 = np.array(F['coherence_dia/data'][:,1,0]) - abs(rho_dia[:,1,0])**2
                
                t = np.array(F['time/data'][:]) * units.au2fs
            
                clrs_index = libra_py.data_visualize.clrs_index[ c2%10 ]
                clr = libra_py.data_visualize.colors[ clrs_index ]
                
                # SE, adi                
                ax[0].plot(t, pop0_se_adi, color=clr, label=F"{method}", lw=5)
                ax[0].plot(t, pop1_se_adi, color=clr, label="", lw=5, ls="--")
                ax[0].plot(t, coh_adi01, color=clr, label="", lw=5, ls='', marker='o')
                ax[0].plot(t, coh_adi10, color=clr, label="", lw=5, ls='', marker='*')

                # SH, adi                
                ax[1].plot(t, pop0_sh_adi, color=clr, label=F"{method}", lw=5)
                ax[1].plot(t, pop1_sh_adi, color=clr, label="", lw=5, ls="--")
                ax[1].plot(t, coh_adi01, color=clr, label="", lw=5, ls='', marker='o')
                ax[1].plot(t, coh_adi10, color=clr, label="", lw=5, ls='', marker='*')
                
                # SE, dia                
                ax[2].plot(t, pop0_se_dia, color=clr, label=F"{method}", lw=5)
                ax[2].plot(t, pop1_se_dia, color=clr, label="", lw=5, ls="--")
                ax[2].plot(t, coh_dia01, color=clr, label="", lw=5, ls='', marker='o')
                ax[2].plot(t, coh_dia10, color=clr, label="", lw=5, ls='', marker='*')

        fig.suptitle(F'MDL = {model} istate = {istate} $q_0$ = {q0} $p_0$ ={p0}', size=38) 
        fig.supxlabel("Time, fs", size=40)    
        plt.legend()
        plt.tight_layout()    
        plt.savefig(F'error-Populations_model_{model}_method_{method}_istate{istate}_q0{q0}_p0{p0}.png')

In [ ]:
#plot_errors("Tully1", iconds1, methods)
#plot_errors("Tully2", iconds1, methods)

In [ ]:
plot_errors2("Tully1", iconds1, methods)
plot_errors2("Tully2", iconds1, methods)

In [ ]:
plot_errors3("Tully1", iconds1, methods)
plot_errors3("Tully2", iconds1, methods)

## 2.2. Metrics of the Methods' Accuracy

Here we consider several metrics to compute method's accuracy score - e.g. via entropies on the difference of the population/coherence deviation from the target values 

- `plot_error_information` - only using population differences $P - P^{ref}$
- `plot_error_information2` - also using coherence scores $\langle |c_i|^2 \ |c_j|^2\rangle - |c_{ij}^{ref}|^2 =  \langle P_i P_j \rangle - |c_{ij}^{ref}|^2$
- `plot_error_information3` - since in the previous metric, we use the products of populations in the section that corresponds to coherences, it is logical to consider not the differences of the populations, but the differences of squared populations  

In [ ]:
def plot_error_information(model, iconds, methods, _bins = 200, _sigma = 0.0075, xbox = 0.05, ybox = 0.1):
    all_df = None
    for c, icond in enumerate(iconds):    
        q0, p0, istate = icond[0], icond[1], icond[2]
    
        #================= Adiabatic SE populations ================    
        prf = F"EXACT-model_{model}-istate_{istate}-q0_{q0}-p0_{p0}"   
    
        f = torch.load(F"{prf}/data.pt")
        t = torch.tensor(f["time"]) * units.au2fs  # to convert to fs
        rho_dia = np.array(torch.tensor(f["rho_dia_all"]))
        rho_adi = np.array(torch.tensor(f["rho_adi_all"]))    
        
        for c2, method in enumerate(methods):
            prf = F"NAMD-model_{model}-istate_{istate}-q0_{q0}-p0_{p0}-method_{method}"
       
            fig, ax = plt.subplots(3, 1, figsize=(16, 18), sharex="col")

            ax[0].set_ylabel("SE, adi")
            ax[1].set_ylabel("SH, adi")
            ax[2].set_ylabel("SE, dia")        

            z0, z1, z2 = [], [], []
            with h5py.File(F"{prf}/mem_data.hdf", 'r') as F:
                pop0_se_adi = abs(np.array(F['se_pop_adi/data'][:,0])) - abs(rho_adi[:,0,0])
                pop0_sh_adi = abs(np.array(F['sh_pop_adi/data'][:,0])) - abs(rho_adi[:,0,0])
                pop0_se_dia = abs(np.array(F['se_pop_dia/data'][:,0])) - abs(rho_dia[:,0,0])
            
                z0 = z0 + list(pop0_se_adi)
                z1 = z2 + list(pop0_sh_adi)
                z2 = z2 + list(pop0_se_dia)
            
                pop0_se_adi = abs(np.array(F['se_pop_adi/data'][:,1])) - abs(rho_adi[:,1,1])
                pop0_sh_adi = abs(np.array(F['sh_pop_adi/data'][:,1])) - abs(rho_adi[:,1,1])
                pop0_se_dia = abs(np.array(F['se_pop_dia/data'][:,1])) - abs(rho_dia[:,1,1])

                z0 = z0 + list(pop0_se_adi)
                z1 = z2 + list(pop0_sh_adi)
                z2 = z2 + list(pop0_se_dia)
            
            print(F"method = {method}, SE, adi:")
            v01, v02, v03 = show_hist(ax[0], z0, _bins, _sigma, xbox, ybox)
        
            print(F"method = {method}, SH, adi:")
            v11, v12, v13 = show_hist(ax[1], z1, _bins, _sigma, xbox, ybox)
        
            print(F"method = {method}, SE, dia:")
            v21, v22, v23 = show_hist(ax[2], z2, _bins, _sigma, xbox, ybox)
            
            df = pd.DataFrame({ "method": [method],
                                "p0": [p0],
                                "q0": [q0],
                                "istate":[istate],
                                "model":[model],
                                "SE, adi, hist inf": [v01],
                                "SE, adi, KDE inf": [v02],
                                "SE, adi, KL inf": [v03],
                                "SH, adi, hist inf": [v11],
                                "SH, adi, KDE inf": [v12],
                                "SH, adi, KL inf": [v13],
                                "SE, dia, hist inf": [v21],
                                "SE, dia, KDE inf": [v22],
                                "SE, dia, KL inf": [v23],
                                "SE, adi, average": [ np.sqrt(np.average( np.array(z0)**2)) ],
                                "SH, adi, average": [ np.sqrt(np.average( np.array(z1)**2)) ],
                                "SE, dia, average": [ np.sqrt(np.average( np.array(z2)**2)) ],
            })
            if all_df is None:
                all_df = df
            else:
                all_df = pd.concat([all_df, df], ignore_index=True)
        
            fig.suptitle(F'MDL = {model} Method = {method} istate = {istate} $q_0$ = {q0} $p_0$ ={p0}', size=38) 
            fig.supxlabel("Population Error", size=40)    
            plt.legend()
            plt.tight_layout()    
            plt.savefig(F'information-Error_model_{model}_method_{method}_istate{istate}_q0{q0}_p0{p0}.png')
            
    return all_df

In [ ]:
def plot_error_information2(model, iconds, methods, _bins = 200, _sigma = 0.0075, xbox = 0.05, ybox = 0.1):
    """
    Add the info on coherences
    """
    all_df = None
    for c, icond in enumerate(iconds):    
        q0, p0, istate = icond[0], icond[1], icond[2]
    
        #================= Adiabatic SE populations ================    
        prf = F"EXACT-model_{model}-istate_{istate}-q0_{q0}-p0_{p0}"   
    
        f = torch.load(F"{prf}/data.pt")
        t = torch.tensor(f["time"]) * units.au2fs  # to convert to fs
        rho_dia = np.array(torch.tensor(f["rho_dia_all"]))
        rho_adi = np.array(torch.tensor(f["rho_adi_all"]))    
        
        for c2, method in enumerate(methods):
            prf = F"NAMD-model_{model}-istate_{istate}-q0_{q0}-p0_{p0}-method_{method}"
       
            fig, ax = plt.subplots(3, 1, figsize=(16, 18), sharex="col")

            ax[0].set_ylabel("SE, adi")
            ax[1].set_ylabel("SH, adi")
            ax[2].set_ylabel("SE, dia")        

            z0, z1, z2 = [], [], []
            with h5py.File(F"{prf}/mem_data.hdf", 'r') as F:
                pop0_se_adi = abs(np.array(F['se_pop_adi/data'][:,0])) - abs(rho_adi[:,0,0])
                pop0_sh_adi = abs(np.array(F['sh_pop_adi/data'][:,0])) - abs(rho_adi[:,0,0])
                pop0_se_dia = abs(np.array(F['se_pop_dia/data'][:,0])) - abs(rho_dia[:,0,0])
            
                z0 = z0 + list(pop0_se_adi)
                z1 = z2 + list(pop0_sh_adi)
                z2 = z2 + list(pop0_se_dia)
            
                pop0_se_adi = abs(np.array(F['se_pop_adi/data'][:,1])) - abs(rho_adi[:,1,1])
                pop0_sh_adi = abs(np.array(F['sh_pop_adi/data'][:,1])) - abs(rho_adi[:,1,1])
                pop0_se_dia = abs(np.array(F['se_pop_dia/data'][:,1])) - abs(rho_dia[:,1,1])

                z0 = z0 + list(pop0_se_adi)
                z1 = z2 + list(pop0_sh_adi)
                z2 = z2 + list(pop0_se_dia)
                
                coh_adi01 = np.array(F['coherence_adi/data'][:,0,1]) - abs(rho_adi[:,0,1])**2
                coh_adi10 = np.array(F['coherence_adi/data'][:,1,0]) - abs(rho_adi[:,1,0])**2
                coh_dia01 = np.array(F['coherence_dia/data'][:,0,1]) - abs(rho_dia[:,0,1])**2
                coh_dia10 = np.array(F['coherence_dia/data'][:,1,0]) - abs(rho_dia[:,1,0])**2
                
                z0 = z0 + list(coh_adi01) + list(coh_adi10)
                z1 = z1 + list(coh_adi01) + list(coh_adi10)
                z2 = z2 + list(coh_dia01) + list(coh_dia10)
                
            
            print(F"method = {method}, SE, adi:")
            v01, v02, v03 = show_hist(ax[0], z0, _bins, _sigma, xbox, ybox)
        
            print(F"method = {method}, SH, adi:")
            v11, v12, v13 = show_hist(ax[1], z1, _bins, _sigma, xbox, ybox)
        
            print(F"method = {method}, SE, dia:")
            v21, v22, v23 = show_hist(ax[2], z2, _bins, _sigma, xbox, ybox)
            
            df = pd.DataFrame({ "method": [method],
                                "p0": [p0],
                                "q0": [q0],
                                "istate":[istate],
                                "model":[model],
                                "SE, adi, hist inf": [v01],
                                "SE, adi, KDE inf": [v02],
                                "SE, adi, KL inf": [v03],
                                "SH, adi, hist inf": [v11],
                                "SH, adi, KDE inf": [v12],
                                "SH, adi, KL inf": [v13],
                                "SE, dia, hist inf": [v21],
                                "SE, dia, KDE inf": [v22],
                                "SE, dia, KL inf": [v23],
                                "SE, adi, average": [ np.sqrt(np.average( np.array(z0)**2)) ],
                                "SH, adi, average": [ np.sqrt(np.average( np.array(z1)**2)) ],
                                "SE, dia, average": [ np.sqrt(np.average( np.array(z2)**2)) ],
            })
            if all_df is None:
                all_df = df
            else:
                all_df = pd.concat([all_df, df], ignore_index=True)
        
            fig.suptitle(F'MDL = {model} Method = {method} istate = {istate} $q_0$ = {q0} $p_0$ ={p0}', size=38) 
            fig.supxlabel("Population Error", size=40)    
            plt.legend()
            plt.tight_layout()    
            plt.savefig(F'information2-Error_model_{model}_method_{method}_istate{istate}_q0{q0}_p0{p0}.png')
            
    return all_df

In [ ]:
def plot_error_information3(model, iconds, methods, _bins = 200, _sigma = 0.0075, xbox = 0.05, ybox = 0.1):
    """
    Add the info on coherences
    """
    all_df = None
    for c, icond in enumerate(iconds):    
        q0, p0, istate = icond[0], icond[1], icond[2]
    
        #================= Adiabatic SE populations ================    
        prf = F"EXACT-model_{model}-istate_{istate}-q0_{q0}-p0_{p0}"   
    
        f = torch.load(F"{prf}/data.pt")
        t = torch.tensor(f["time"]) * units.au2fs  # to convert to fs
        rho_dia = np.array(torch.tensor(f["rho_dia_all"]))
        rho_adi = np.array(torch.tensor(f["rho_adi_all"]))    
        
        for c2, method in enumerate(methods):
            prf = F"NAMD-model_{model}-istate_{istate}-q0_{q0}-p0_{p0}-method_{method}"
       
            fig, ax = plt.subplots(3, 1, figsize=(16, 18), sharex="col")

            ax[0].set_ylabel("SE, adi")
            ax[1].set_ylabel("SH, adi")
            ax[2].set_ylabel("SE, dia")        

            z0, z1, z2 = [], [], []
            with h5py.File(F"{prf}/mem_data.hdf", 'r') as F:
                pop0_se_adi = abs(np.array(F['se_pop_adi/data'][:,0]))**2 - abs(rho_adi[:,0,0])**2
                pop0_sh_adi = abs(np.array(F['sh_pop_adi/data'][:,0]))**2 - abs(rho_adi[:,0,0])**2
                pop0_se_dia = abs(np.array(F['se_pop_dia/data'][:,0]))**2 - abs(rho_dia[:,0,0])**2
            
                z0 = z0 + list(pop0_se_adi)
                z1 = z2 + list(pop0_sh_adi)
                z2 = z2 + list(pop0_se_dia)
            
                pop1_se_adi = abs(np.array(F['se_pop_adi/data'][:,1]))**2 - abs(rho_adi[:,1,1])**2
                pop1_sh_adi = abs(np.array(F['sh_pop_adi/data'][:,1]))**2 - abs(rho_adi[:,1,1])**2
                pop1_se_dia = abs(np.array(F['se_pop_dia/data'][:,1]))**2 - abs(rho_dia[:,1,1])**2

                z0 = z0 + list(pop1_se_adi)
                z1 = z2 + list(pop1_sh_adi)
                z2 = z2 + list(pop1_se_dia)
                
                coh_adi01 = np.array(F['coherence_adi/data'][:,0,1]) - abs(rho_adi[:,0,1])**2
                coh_adi10 = np.array(F['coherence_adi/data'][:,1,0]) - abs(rho_adi[:,1,0])**2
                coh_dia01 = np.array(F['coherence_dia/data'][:,0,1]) - abs(rho_dia[:,0,1])**2
                coh_dia10 = np.array(F['coherence_dia/data'][:,1,0]) - abs(rho_dia[:,1,0])**2
                
                z0 = z0 + list(coh_adi01) + list(coh_adi10)
                z1 = z1 + list(coh_adi01) + list(coh_adi10)
                z2 = z2 + list(coh_dia01) + list(coh_dia10)
                
            
            print(F"method = {method}, SE, adi:")
            v01, v02, v03 = show_hist(ax[0], z0, _bins, _sigma, xbox, ybox)
        
            print(F"method = {method}, SH, adi:")
            v11, v12, v13 = show_hist(ax[1], z1, _bins, _sigma, xbox, ybox)
        
            print(F"method = {method}, SE, dia:")
            v21, v22, v23 = show_hist(ax[2], z2, _bins, _sigma, xbox, ybox)
            
            df = pd.DataFrame({ "method": [method],
                                "p0": [p0],
                                "q0": [q0],
                                "istate":[istate],
                                "model":[model],
                                "SE, adi, hist inf": [v01],
                                "SE, adi, KDE inf": [v02],
                                "SE, adi, KL inf": [v03],
                                "SH, adi, hist inf": [v11],
                                "SH, adi, KDE inf": [v12],
                                "SH, adi, KL inf": [v13],
                                "SE, dia, hist inf": [v21],
                                "SE, dia, KDE inf": [v22],
                                "SE, dia, KL inf": [v23],
                                "SE, adi, average": [ np.sqrt(np.average( np.array(z0)**2)) ],
                                "SH, adi, average": [ np.sqrt(np.average( np.array(z1)**2)) ],
                                "SE, dia, average": [ np.sqrt(np.average( np.array(z2)**2)) ],
            })
            if all_df is None:
                all_df = df
            else:
                all_df = pd.concat([all_df, df], ignore_index=True)
        
            fig.suptitle(F'MDL = {model} Method = {method} istate = {istate} $q_0$ = {q0} $p_0$ ={p0}', size=38) 
            fig.supxlabel("Population Error", size=40)    
            plt.legend()
            plt.tight_layout()    
            plt.savefig(F'information3-Error_model_{model}_method_{method}_istate{istate}_q0{q0}_p0{p0}.png')
            
    return all_df

In [ ]:
# show more rows/cols
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)

# control precision
pd.set_option("display.precision", 3)

### Tully 1

In [ ]:
Tully1_inf1 = plot_error_information("Tully1", iconds1, methods, _bins = 200, _sigma = 0.0075, xbox = 0.05, ybox = 0.1)

In [ ]:
display(Tully1_inf1)

In [ ]:
Tully1_inf2 = plot_error_information2("Tully1", iconds1, methods, _bins = 200, _sigma = 0.0075, xbox = 0.05, ybox = 0.1)

In [ ]:
display(Tully1_inf2)

In [ ]:
Tully1_inf3 = plot_error_information3("Tully1", iconds1, methods, _bins = 200, _sigma = 0.0075, xbox = 0.05, ybox = 0.1)

In [ ]:
display(Tully1_inf3)

### Tully 2

In [ ]:
Tully2_inf1 = plot_error_information("Tully2", iconds1, methods, _bins = 200, _sigma = 0.0075, xbox = 0.05, ybox = 0.1)

In [ ]:
display(Tully2_inf1)

In [ ]:
Tully2_inf2 = plot_error_information2("Tully2", iconds1, methods, _bins = 200, _sigma = 0.0075, xbox = 0.05, ybox = 0.1)

In [ ]:
display(Tully2_inf2)

In [ ]:
Tully2_inf3 = plot_error_information3("Tully2", iconds1, methods, _bins = 200, _sigma = 0.0075, xbox = 0.05, ybox = 0.1)

In [ ]:
display(Tully2_inf3)

Combine data and save:

In [ ]:
data_inf1 = pd.concat([Tully1_inf1, Tully2_inf1], ignore_index=True)
data_inf1.to_csv("metrics_Tully_1_2_inf1.csv", index=False)

In [ ]:
data_inf2 = pd.concat([Tully1_inf2, Tully2_inf2], ignore_index=True)
data_inf2.to_csv("metrics_Tully_1_2_inf2.csv", index=False)

In [ ]:
data_inf3 = pd.concat([Tully1_inf3, Tully2_inf3], ignore_index=True)
data_inf3.to_csv("metrics_Tully_1_2_inf3.csv", index=False)